### pip install & import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

### 최종데이터 불러오기

In [68]:
# movie_final_회귀 = pd.read_csv('../3_merge_top300/movie_final_reg.csv',encoding='utf-8')
movie_final_분류 = pd.read_csv('../3_merge_top300/movie_final_cls.csv',encoding='utf-8')

## 원핫인코딩 

In [69]:
# 원핫인코딩

# movie_labels = pd.get_dummies(movie_final,columns = ['국적','장르','등급','계절'])

In [70]:
# 독립 변수와 종속 변수 나눔

# 독립 = movie_DL[['감독_흥행', '배급사_흥행', '국적_기타', '국적_독일', '국적_미국',
#        '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF', '장르_가족', '장르_공연',
#        '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬',
#        '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_성인물(에로)', '장르_스릴러',
#        '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름']] 
# 종속 = movie_DL[['전국 관객수']] 

# 스케일링

In [71]:
from sklearn.preprocessing import RobustScaler

# 객체생성
robust_scaler = RobustScaler()

# *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
A_n = robust_scaler.fit_transform(movie_final_분류['전국 스크린수'].values.reshape(-1,1))
movie_final_분류.insert(0, '전국 스크린수_scaled', A_n)
# movie_final_분류.drop(['전국 스크린수'], axis=1, inplace=False)

A_n1 = robust_scaler.fit_transform(movie_final_분류['총 관객수'].values.reshape(-1,1))
movie_final_분류.insert(0, '총 관객수_scaled', A_n1)
# movie_final_분류.drop(['총 관객수'], axis=1, inplace=False)

In [72]:
movie_final_분류.columns

Index(['총 관객수_scaled', '전국 스크린수_scaled', 'Unnamed: 0', '순번', '영화명', '감독',
       '배급사', '개봉일', '영화형태', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액',
       '서울 관객수', '장르', '등급', '영화구분', '연도', '월', '계절', '총 관객수', '코로나', '출연',
       '평점', '감독_흥행', '배급사_흥행', '주연배우_흥행'],
      dtype='object')

## 라벨링

In [73]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()

movie_labels = movie_final_분류.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final_분류['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final_분류['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final_분류['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final_분류['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final_분류['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final_분류['계절'])
movie_labels.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_labels.head(3)


,총 관객수_scaled,전국 스크린수_scaled,순번,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,...,연도,월,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행
0,0.027876,1.397641,1,명량,김한민,씨제이이엔엠,2014-07-30,0,7,1587,...,2014,7,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20
1,0.216263,2.056445,2,극한직업,이병헌,씨제이이엔엠,2019-01-23,0,7,1978,...,2019,1,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16
2,0.105881,1.945240,3,신과함께-죄와 벌,김용화,롯데엔터테인먼트,2017-12-20,0,7,1912,...,2017,12,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40


In [74]:
# movie_labels[movie_labels['연도']!=2023]

### 데이터분배
1) movie_labels_etc : train과 val data   
2) movie_labels_2023 : test data

In [75]:
# movie_labels_2023 = movie_labels[movie_labels['연도']==2023]
# movie_labels_etc = movie_labels[movie_labels['연도']!=2023]
# display(movie_labels_etc.head(),
# movie_labels_2023.head())

### 변수 정리   
1. 사용하지 않는 컬럼 제거
2. 독립변수와 종속변수 설정

In [76]:
# # 사용하지 않는 컬럼 제거

# not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

# movie_DL = movie_labels_etc.drop(not_used, axis=1)      # 2003~2022 데이터 main
# movie_DL2 = movie_labels_2023.drop(not_used, axis=1)    # 2023년 test


In [77]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

movie_DL = movie_labels.drop(not_used, axis=1)
movie_DL.head()

,총 관객수_scaled,전국 스크린수_scaled,순번,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,연도,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행
0,0.027876,1.397641,1,7,1587,135748398910,17613682,"33,121,225,810","4,163,666",9,1,2014,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20
1,0.216263,2.056445,2,7,1978,139647979516,16264944,"31,858,660,536","3,638,287",16,1,2019,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16
2,0.105881,1.945240,3,7,1912,115698654137,14410754,"27,530,825,087","3,346,172",17,0,2017,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40
3,0.027876,0.351306,4,7,966,110828014630,14245998,"25,842,519,330","3,233,946",4,0,2014,1,215068958,0,"['황정민', '김윤진', '오달수', '정진영']",9.02,10,10,17
4,0.216263,3.500421,5,2,2835,122182694160,13934592,"33,577,136,860","3,597,963",13,0,2019,2,226678777,0,"['로버트 다우니 주니어', '크리스 에반스', '크리스 헴스워스', '마크 러팔로...",9.38,10,10,106


In [78]:
# 국적 => 국내 vs 해외
# 관객비율.. 2023년 데이터 총 관객수 몰라....
# 비율 말고 총 관객수를 쓰자
# 총관객수 defqult를 2022년으로 해서 2023년 총관객수 채우면 어때?

In [79]:
# # 독립 변수와 종속 변수 나눔

# 종속 = movie_DL[['전국 관객수']].copy()
# 종속['전국 관객수'] = 종속['전국 관객수'] / 1000 

# 종속_2023 = movie_DL2[['전국 관객수']].copy()
# 종속_2023['전국 관객수'] = 종속_2023['전국 관객수'] / 1000 


# 독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# 독립_2023 = movie_DL2[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# display(독립.head(), 독립_2023.head())

In [80]:
독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

종속 = movie_DL[['전국 관객수']].copy()

display(독립.info(), 종속.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   장르              708 non-null    int32  
 1   등급              708 non-null    int32  
 2   감독_흥행           708 non-null    int64  
 3   총 관객수_scaled    708 non-null    float64
 4   배급사_흥행          708 non-null    int64  
 5   주연배우_흥행         708 non-null    int64  
 6   계절              708 non-null    int32  
 7   전국 스크린수_scaled  708 non-null    float64
 8   연도              708 non-null    int64  
 9   코로나             708 non-null    int64  
dtypes: float64(2), int32(3), int64(5)
memory usage: 47.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   전국 관객수  708 non-null    int64
dtypes: int64(1)
memory usage: 5.7 KB


None

None

## train, test 데이터 나누기

In [81]:
# train / test 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(독립, 종속, test_size=0.20, random_state=42)

In [82]:
# train / val 나누기
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.20, random_state=42)
print(len(train_input), len(val_input), len(test_input))

452 114 142


In [83]:
# # train, validation 데이터 나눔

# from sklearn.model_selection import train_test_split
# train_input, test_input, train_target, test_target = train_test_split(독립,종속,test_size=0.25,random_state=42)

## target 클래스 분류하기

In [84]:
display(
    len(train_target[train_target['전국 관객수']<1000000]),
    len(train_target[(train_target['전국 관객수']>1000000) & (train_target['전국 관객수']<1400000)]),
    len(train_target[(train_target['전국 관객수']>1400000) & (train_target['전국 관객수']<2000000)]),
    len(train_target[(train_target['전국 관객수']>2000000) & (train_target['전국 관객수']<3000000)]),
    len(train_target[(train_target['전국 관객수']>3000000) & (train_target['전국 관객수']<5000000)]),
    len(train_target[train_target['전국 관객수']>5000000])
)

77

101

102

99

83

69

In [85]:
# movie_resize_분류['코로나'] = np.where((movie_resize_분류['연도'].values >=2020) & (movie_resize_분류['연도'].values <=2022) , 1, 0)

import numpy as np

train_target['분류클래스'] = np.where((train_target['전국 관객수'].values<1000000), 1, 
                                 np.where((train_target['전국 관객수'].values>1000000) & (train_target['전국 관객수'].values<1400000), 2,
                                          np.where((train_target['전국 관객수'].values>1400000) & (train_target['전국 관객수'].values<2000000), 3,
                                                   np.where((train_target['전국 관객수'].values>2000000) & (train_target['전국 관객수'].values<3000000), 4,
                                                            np.where((train_target['전국 관객수'].values>3000000) & (train_target['전국 관객수'].values<5000000), 5, 6))))
)
train_target['분류클래스'].unique()
train_target[['전국 관객수','분류클래스']]
train_target['분류클래스'].value_counts()

3    102
2    101
4     99
5     83
1     77
6     69
Name: 분류클래스, dtype: int64

In [40]:
val_target['분류클래스'] = np.where((val_target['전국 관객수'].values<1000000), 1, 
                                 np.where((val_target['전국 관객수'].values>1000000) & (val_target['전국 관객수'].values<1400000), 2,
                                          np.where((val_target['전국 관객수'].values>1400000) & (val_target['전국 관객수'].values<2000000), 3,
                                                   np.where((val_target['전국 관객수'].values>2000000) & (val_target['전국 관객수'].values<3000000), 4,
                                                            np.where((val_target['전국 관객수'].values>3000000) & (val_target['전국 관객수'].values<5000000), 5, 6))))
)
val_target['분류클래스'].unique()
# val_target[['전국 관객수','분류클래스']]

array([5, 1, 2, 3, 6, 4])

In [41]:
test_target['분류클래스'] = np.where((test_target['전국 관객수'].values<1000000), 1, 
                                 np.where((test_target['전국 관객수'].values>1000000) & (test_target['전국 관객수'].values<1400000), 2,
                                          np.where((test_target['전국 관객수'].values>1400000) & (test_target['전국 관객수'].values<2000000), 3,
                                                   np.where((test_target['전국 관객수'].values>2000000) & (test_target['전국 관객수'].values<3000000), 4,
                                                            np.where((test_target['전국 관객수'].values>3000000) & (test_target['전국 관객수'].values<5000000), 5, 6))))
)
test_target['분류클래스'].unique()
# test_target[['전국 관객수','분류클래스']]

array([5, 4, 2, 3, 6, 1])

In [42]:
# # 값 스케일링

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_scaled = ss.transform(train_input)
# test_scaled = ss.transform(test_input)

# 딥러닝 

In [45]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[10])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)
dropout = tf.keras.layers.Dropout(rate=0.3)(H)

Y = tf.keras.layers.Dense(6, activation='softmax')(dropout)
model = tf.keras.models.Model(X,Y)
#sgd = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.8,beta_2=0.85)
# sgd = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.7)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [47]:
# 훈련
history = model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(val_input,val_target))

Epoch 1/100


ValueError: in user code:

    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Playdata\AppData\Roaming\Python\Python39\site-package

In [58]:
# 확인용도
model.predict(train_input[:5])

1/1 [==============================] - 0s 123ms/step


array([[131.46555 ],
       [127.31804 ],
       [128.48778 ],
       [127.678375],
       [124.575195]], dtype=float32)

In [59]:
# 여기랑 비슷한지 보기
train_target[:5]

,전국 관객수
952,20.175
335,56.282
933,21.071
1354,10.854
1007,18.501


### test data 확인용도

In [61]:
# 확인용도

# 오차율 몇 %로...?
pred = model.predict(독립_2023).round(0)
# 여기랑 비슷한지 보기
종속_2023['pred'] = pred
종속_2023['오차율'] = (abs(종속_2023['전국 관객수'] - 종속_2023['pred'])) / 종속_2023['전국 관객수'] * 100
len(종속_2023.loc[종속_2023['오차율']<20])

1/1 [==============================] - 0s 29ms/step


0

In [42]:
종속_2023

,전국 관객수,pred,오차율
398,1720.865,2441.0,41.847269
485,1376.493,4026.0,192.482417
690,829.268,1706.0,105.723602
